# _Berkeley Political Review_ Data Visualization Template

## US Map

_Fall 2019_

In [164]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import bpr

%matplotlib inline

import chart_studio.plotly as pl
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [165]:
#DON'T CHANGE THIS CELL, JUST RUN IT

f = open("credentials.txt")

u = f.readline()
plotly_user = str(u[:-1])

k = f.readline()
plotly_api_key = str(k)

In [166]:
#DON'T CHANGE THIS CELL, JUST RUN IT
pl.sign_in(username = plotly_user, api_key = plotly_api_key)

In [173]:
ga_gov_election = pd.read_csv('Data/GeorgiaGovernorsResults.csv')
ga_gov_election.head()

,FIPS,County,Abrams,Kemp,Metz
0,13001,Appling,1358,5428,23
1,13003,Atkinson,637,1876,9
2,13005,Bacon,489,3321,20
3,13007,Baker,535,753,5
4,13009,Baldwin,7793,7735,108


In [176]:
ga_gov_election.loc[ga_gov_election['County'] == 'De Kalb', 'County'] = 'DeKalb'

In [177]:
ga_gov_election['Percent_Vote'] = 100 * ga_gov_election['Kemp'] / (ga_gov_election['Kemp'] + ga_gov_election['Abrams'] + ga_gov_election['Metz'])
ga_gov_election.head()

,FIPS,County,Abrams,Kemp,Metz,Percent_Vote
0,13001,Appling,1358,5428,23,79.718020
1,13003,Atkinson,637,1876,9,74.385408
2,13005,Bacon,489,3321,20,86.710183
3,13007,Baker,535,753,5,58.236659
4,13009,Baldwin,7793,7735,108,49.469174


In [178]:
ga_gov_election['Margin'] = (100 * ga_gov_election['Kemp'] / (ga_gov_election['Kemp'] + ga_gov_election['Abrams'] + ga_gov_election['Metz'])) - (100 * ga_gov_election['Abrams'] / (ga_gov_election['Kemp'] + ga_gov_election['Abrams'] + ga_gov_election['Metz']))
ga_gov_election[ga_gov_election['County'] == 'Fulton']

,FIPS,County,Abrams,Kemp,Metz,Percent_Vote,Margin
59,13121,Fulton,306589,112991,4208,26.662152,-45.682747


In [179]:
ga_gov_election[(ga_gov_election['Margin'] < 5) & (ga_gov_election['Margin'] > -5)]

,FIPS,County,Abrams,Kemp,Metz,Percent_Vote,Margin
4,13009,Baldwin,7793,7735,108,49.469174,-0.370939
16,13033,Burke,4269,4410,42,50.567595,1.616787
110,13225,Peach,4966,5432,47,52.005744,4.461465
128,13261,Sumter,5360,5149,47,48.777946,-1.998863
149,13303,Washington,4012,4128,29,50.532501,1.420002


In [181]:
ga_gov_election[ga_gov_election['County'] == 'DeKalb']

,FIPS,County,Abrams,Kemp,Metz,Percent_Vote,Margin
43,13089,DeKalb,261042,48923,2776,15.643296,-67.825773


In [182]:
bins = list(np.mgrid[min(ga_gov_election['Percent_Vote']):max(ga_gov_election['Percent_Vote']):28j])
bins[13] = 50.0
bins = [0] + bins
bins

[0,
 11.785884700472824,
 14.734231100811163,
 17.682577501149503,
 20.630923901487844,
 23.579270301826185,
 26.527616702164522,
 29.475963102502863,
 32.4243095028412,
 35.37265590317954,
 38.321002303517886,
 41.269348703856224,
 44.21769510419456,
 47.1660415045329,
 50.0,
 53.062734305209574,
 56.011080705547926,
 58.95942710588626,
 61.9077735062246,
 64.85611990656294,
 67.80446630690128,
 70.75281270723961,
 73.70115910757796,
 76.64950550791629,
 79.59785190825464,
 82.54619830859298,
 85.49454470893131,
 88.44289110926965,
 91.39123750960799]

In [211]:
fig = ff.create_choropleth(
    fips = ga_gov_election['FIPS'],
    values = ga_gov_election['Percent_Vote'], 
    scope=['GA'],
    #colorscale=bpr.safe_likely_leans_tossup,
    #simplify_county=0,
    
    plot_bgcolor='#FFFFFF',
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5}, 
    colorscale = bpr.safe_likely_leans_extra,
    binning_endpoints= bins,
    title='Georgia 2018 Gubernatorial Election Results',
    
    
    simplify_county=0.002
)

fig.update_layout(
    dragmode = False,
    clickmode = 'none',
    showlegend = False,
)

fig.show()

In [212]:
pl.iplot(fig, filename="GA-Gubernatorial-Race-Results")

In [213]:
hover_ix, hover = [(ix, t) for ix, t in enumerate(fig['data']) if t.text][0]

# mismatching lengths indicates bug
if len(hover['text']) != len(ga_gov_election):

    ht = pd.Series(hover['text'])

    no_dupe_ix = ht.index[~ht.duplicated()]

    hover_x_deduped = np.array(hover['x'])[no_dupe_ix]
    hover_y_deduped = np.array(hover['y'])[no_dupe_ix]

    new_hover_x = [x if type(x) == float else x[0] for x in hover_x_deduped]
    new_hover_y = [y if type(y) == float else y[0] for y in hover_y_deduped]

    fig['data'][hover_ix]['text'] = ht.drop_duplicates()
    fig['data'][hover_ix]['x'] = new_hover_x
    fig['data'][hover_ix]['y'] = new_hover_y
    
fig.show()

In [214]:
ga_gov_election[ga_gov_election['County'] == 'Fulton']['Kemp'].values[0]

112991

In [215]:
#DO NOT RUN THIS CODE

# Get hover trace
hover_scatter = [scatt for scatt in fig.data if scatt.mode == 'markers'][0]

# Update text to include only county (Not state, FIPS, ec.)
import re
def extract_county(text_val):
    county = re.search(r'(?<=County: )(.*?)(?=<br>)', text_val).group(0)
    #print(county)
    #print(ga_gov_election[ga_gov_election['County'] == county])
    k = str(ga_gov_election[ga_gov_election['County'] == county]['Kemp'].values[0])
    a = str(ga_gov_election[ga_gov_election['County'] == county]['Abrams'].values[0])
    m = str(ga_gov_election[ga_gov_election['County'] == county]['Metz'].values[0])
    return county + ' County' + '<br>' + 'Kemp: ' + k + '<br>' + 'Abrams: ' + a + '<br>' + 'Metz: ' + m

hover_scatter.text = [extract_county(text_val) for text_val in hover_scatter.text]

# Show text
#hover_scatter.mode = 'markers+text'

#fig.show()

In [216]:
fig.show()

In [217]:
pl.iplot(fig, filename="GA-Gubernatorial-Race-Results")

In [218]:
small_margin = ga_gov_election[(ga_gov_election['Margin'] < 5) & (ga_gov_election['Margin'] > -5)]

In [228]:
fig = ff.create_choropleth(
    fips = small_margin['FIPS'],
    values = small_margin['Percent_Vote'], 
    scope=['GA'],
    #colorscale=bpr.safe_likely_leans_tossup,
    #simplify_county=0,
    
    plot_bgcolor='#FFFFFF',
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5}, 
    colorscale = bpr.safe_likely_leans_extra,
    binning_endpoints= bins,
    title='Georgia 2018 Gubernatorial Election - Counties with Margin < 5%',
    
    
    simplify_county=0.002
)

fig.update_layout(
    dragmode = False,
    clickmode = 'none',
    showlegend = False,
)

fig.show()

In [229]:
hover_ix, hover = [(ix, t) for ix, t in enumerate(fig['data']) if t.text][0]

# mismatching lengths indicates bug
if len(hover['text']) != len(small_margin):

    ht = pd.Series(hover['text'])

    no_dupe_ix = ht.index[~ht.duplicated()]

    hover_x_deduped = np.array(hover['x'])[no_dupe_ix]
    hover_y_deduped = np.array(hover['y'])[no_dupe_ix]

    new_hover_x = [x if type(x) == float else x[0] for x in hover_x_deduped]
    new_hover_y = [y if type(y) == float else y[0] for y in hover_y_deduped]

    fig['data'][hover_ix]['text'] = ht.drop_duplicates()
    fig['data'][hover_ix]['x'] = new_hover_x
    fig['data'][hover_ix]['y'] = new_hover_y
    
fig.show()

In [230]:
#DO NOT RUN THIS CODE

# Get hover trace
hover_scatter = [scatt for scatt in fig.data if scatt.mode == 'markers'][0]

# Update text to include only county (Not state, FIPS, ec.)
import re
def extract_county(text_val):
    county = re.search(r'(?<=County: )(.*?)(?=<br>)', text_val).group(0)
    #print(county)
    #print(ga_gov_election[ga_gov_election['County'] == county])
    margin = str(round(ga_gov_election[ga_gov_election['County'] == county]['Margin'].values[0], 4))
    
    return county + ' County' + '<br>' + 'Margin: ' + margin + '%'

hover_scatter.text = [extract_county(text_val) for text_val in hover_scatter.text]

# Show text
#hover_scatter.mode = 'markers+text'

#fig.show()

In [231]:
fig.show()

In [232]:
pl.iplot(fig, filename="GA-Gov-Margin-5")